# Using quantum cloud services API with Tangelo

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/goodchemistryco/Tangelo/blob/develop/examples/linq/2.qpu_connection.ipynb)

This notebook elaborates on how the `tangelo.linq` module can facilitate hardware experiments by integrating the API provided by some hardware providers (IonQ, Honeywell...) or broader quantum cloud services providers such as Azure Quantum or Braket. We essentially provide convenience wrappers to these APIs: the API of your favorite cloud provider is still called underneath, using your own credentials.

This notebook should be relevant to Tangelo users that prefer to install all the dependencies on their machine, use their own credentials, and be billed directly by the target quantum cloud service(s).

This approach is an alternative to using [QEMIST Cloud](https://goodchemistry.com/qemist-cloud/), which we intend to make a single and convenient entry point to reach many different platforms. What's pretty cool with this other option is that users do not have to set up the environment required by the quantum cloud(s) of their choice (e.g Braket, Azure Quantum, etc) or even an account with those services. A QEMIST Cloud account provides a single entry point enabling you to reach all of them, and pay with your QEMIST Cloud credits.


## Table of contents
* [1. IonQ Rest API](#1)
    * [1.1 References](#1.1)
    * [1.2 Environment](#1.2)
    * [1.3 Quantum Circuit](#1.3)
    * [1.4 IonQConnection class](#1.4)
* [2. Azure Quantum](#2)
* [3. Braket](#3)

---

## Requirements

- Tangelo needs to be installed in your environment. The cell below does this installation for you if it is not found.
- Ensure you have valid ID tokens and logins for the services of your choice (IonQ, Braket...), and that your environment is ready to use them (e.g install their SDK and dependencies according to the instructions they provide)

In [1]:
# Installation of tangelo if not already installed.
try:
    import tangelo
except ModuleNotFoundError:
    !pip install git+https://github.com/goodchemistryco/Tangelo.git@develop  --quiet

## 1. IonQ Rest API <a class="anchor" id="1"></a>


### 1.1 References <a class="anchor" id="1.1"></a>

Tangelo provides a few Python wrappers around the REST API provided by IonQ, to facilitate job submission and monitoring. The most up-to-date list of supported features and information regarding the IonQ API is available at https://docs.ionq.co

### 1.2 Environment <a class="anchor" id="1.2"></a>

Users allowed to use the IonQ API should have an ID token, e.g a string of alphanumeric characters and dashes, which can be obtained through the [IonQ dashboard](https://cloud.ionq.com/). Users need to set their `IONQ_APIKEY` environment variable to this value; here are two ways to do it:

- in your terminal (`export IONQ_APIKEY=<value>`)
- or, in your Python script (`os.environ['IONQ_APIKEY'] = <value>`)

Here's an example of what it could look like for you (make sure you use a valid key, or you'll get an "unauthorized" error :) )

In [2]:
import os
os.environ["IONQ_APIKEY"] = '2T14z1YQEzMLCwuYM110oXPDT2h850E4'

### 1.3 Quantum circuit <a class="anchor" id="1.3"></a>

IonQ uses a JSON representation for quantum circuits, which `tangelo.linq` can generate from an abstract circuit, using the `translator` module. IonQ also offers partial support for other common formats (OpenQASM, ...). For this example, we'll work with their JSON format, which is pretty much nested lists and dictionaries. For more detailed and up-to-date information, check out their [documentation](https://docs.ionq.com/#tag/quantum_programs).

Below, we show how users can produce a simple quantum circuit in the JSON format:

In [3]:
from tangelo.linq import Circuit, Gate, translate_circuit

circ1 = Circuit([Gate("H", 0), Gate("X", 1)])
json_circ1 = translate_circuit(circ1, target="ionq")
print(json_circ1)

{'qubits': 2, 'circuit': [{'gate': 'h', 'targets': [0]}, {'gate': 'x', 'targets': [1]}]}


Our convenience wrappers below do not require users to translate their circuit into the IonQ format: providing a circuit in the Tangelo format is enough.

### 1.4 IonQConnection class <a class="anchor" id="1.4"></a>

The `IonQConnection` class encapsulates a collection of wrappers to the IonQ REST API. Internally, it stores information about the endpoint and the authorization header, containing your identification token. This class only is instantiated succesfully if your ID token has been set properly.

More generally speaking, all calls to the REST API are checked for errors, and would return the IonQ error message corresponding to the unsuccessful request.

In [4]:
from tangelo.linq.qpu_connection import IonQConnection

ionq_connection = IonQConnection()

{'error': 'Unauthorized', 'statusCode': 401}


RuntimeError: Error returned by IonQ API :
Unauthorized

#### Backend info

Tangelo's `get_backend_info` method allows us to return some information about all the devices listed on IonQ's platform. Tangelo currently returns this info inside a `pandas.DataFrame` object, to help with visualization. 

In [ ]:
res = ionq_connection.get_backend_info()
res # to display the dataframe neatly in the notebook

This information can help users to filter or sort devices according to their needs. For example, filtering out devices who do not have enough qubits for the target experiment, as well as the unavailable devices.

We can also retrieve "characterizations": a snapshot of the IonQ platform's performance at a moment in time. We can use the `get_characterization` method with either a backend string (ex: `qpu.harmony` or `simulator`) or a  characterization url, if available (see dataframe above).

In [ ]:
charac_dict = ionq_connection.get_characterization('qpu.s11')
print(charac_dict['fidelity'])
print(charac_dict['timing'])

This information can help users having a better understanding of the capabilities of a device, and anticipate its performance on their usecases.

Please check [IonQ's documentation](https://docs.ionq.com/#tag/characterizations) to confirm what these quantities mean, and the units in which they are expressed in.

#### Job submission

Job submission can be achieved through the `job_submit` method, which attempts to submit a job on a backend (simulator or QPU), and returns a job ID if submission was succesful.

This method takes input arguments that need to be provided by the user:

- the target backend (a string such as 'simulator' or one that refers to a QPU)
- the quantum circuit (in Tangelo format)
- the number of shots required
- a name for your job
- any other option as key arguments (see source code and IonQ documentation)

Assuming a valid API key, we can then submit a simple job targeting their statevector simulator. The status of the job may be in various states (queued, ready, running ...).

In [ ]:
job_id = ionq_connection.job_submit('simulator', circ1, 100, 'test_json_job')

#### Job history and job info

Users can access their job history and info through the `job_get_history` and `job_status` methods, shown as below. But IonQ also provides an [online dashboard](https://cloud.ionq.com/), which may be more convenient to you.

Depending on the timing of your REST requests, the job info may differ widely, from a failed job to a completed job with results included.

In [ ]:
job_status = ionq_connection.job_status(job_id)
print(job_status)

The output of `job_get_history` should at least feature the job we just submitted, and can also show a number of previous jobs run under your account. The output is a `pandas` dataframe, in order to facilitate parsing of information:

In [ ]:
job_history = ionq_connection.job_get_history()
job_history[:5] # Here we only display info of the last 5 jobs run

#### Job results

The method `job_results` provides a wrapper to a blocking call, querying for the state of the target job at regular intervals, attempting to retrieve the results. If the job has successfully completed, this method returns a dictionary with bitstring keys (ex: `01`, `11`...)

IonQ raw results use a "most-significant qubit first" representation, encoded as an integer, but Tangelo returns them as a bitstring in least-significant order (e.g we read left-to-right), to stay consistant with its own format and what is common across other cloud services.

For our example circuit, IonQ's raw results would return `{'2': 0.5, '3': 0.5}` with an exact simulator.
- 2 is '10' in binary, indicating $q_0$ mesured in state $|0\rangle$, and $q_1$ in state $|1\rangle$
- 3 is '11' in binary, indicating $q_0$ mesured in state $|1\rangle$, and $q_1$ in state $|1\rangle$

Tangelo returns `{'01': 0.5, '11': 0.5}`.

In [ ]:
results = ionq_connection.job_results(job_id)
print(results)

#### Job cancel / delete

A wrapper called `job_cancel` provides a method to cancel before execution (if the job hasn't run yet), or delete a job from the history. The cell below cancels / deletes the previously-submitted job: it therefore should not appear in the history anymore.

In [ ]:
ionq_connection.job_cancel(job_id)
job_history = ionq_connection.job_get_history()
job_history

## 2. Azure Quantum <a class="anchor" id="2"></a>

Though this package does not currently provide a way to directly submit jobs through Microsoft's Azure Quantum cloud services, the `translate_circuit` function (with the `target="qdk"` option) in the `translator` module can parse an abstract circuit and generate Q# code that can be written to file. 

Azure quantum also supports circuits in a number of other formats (`cirq`, `qiskit`, IonQ...), which means any of your favorite translate function from `tangelo.linq` can come in handy: check out their [documentation](https://docs.microsoft.com/en-us/azure/quantum/concepts-circuits) for the most reliable information.

Our generated Q# code is compatible with both the local QDK simulator (good for testing before submitting to an actual QPU) or by Azure Quantum. Submission through Azure Quantum will require the user to have an account on Azure, install the local CLI and Python dependencies.

For an example of how one can use this package to first generate circuits, and then submit jobs through Azure quantum, please look into the `example/qsharp` folder of this package.

## 3. Braket <a class="anchor" id="3"></a>

Tangelo currently offers to generate a quantum circuit in the Braket format using the `translate_circuit` function (with the `target="braket"` option) from the translator module, and may provide convenience wrappers later on around Braket's API. Once you have a circuit in Braket's format, you can pretty much do anything you want using their SDK.

In [5]:
abstract_circuit = Circuit([Gate('H',0), Gate('CNOT', target=1, control=0)])
braket_circuit = translate_circuit(abstract_circuit, target="braket")

print(braket_circuit)

T  : |0|1|
          
q0 : -H-C-
        | 
q1 : ---X-

T  : |0|1|


Users can use the [Braket python SDK](https://github.com/aws/amazon-braket-sdk-python/) provided by Amazon (`amazon-braket-sdk`) to submit experiments to backends available in the Braket cloud and retrieve results, or simply run them using the [LocalSimulator](https://docs.aws.amazon.com/braket/latest/developerguide/braket-get-started-run-circuit.html) that comes with the SDK.

Braket relies on you installing the AWS CLI, and assumes you have an IAM user with proper permissions (Braket, S3 buckets). The Braket services can also be accessed through your web browser, which gives you access to managed python notebooks.

> Signing up for [QEMIST Cloud](https://goodchemistry.com/qemist-cloud/) allows you to otherwise use a single entry-point, does not require you to install Braket dependencies, and pay with your QEMIST Cloud credits.

This simple [Braket example](https://github.com/aws/amazon-braket-sdk-python/blob/main/examples/bell.py) shows how you can retrieve measurement counts. Bitstrings are read left-to-right (first is qubit 0, then qubit 1...).

The `measurement_counts` method returns an object of the built-in Python `Counter` class.

In [6]:
# Illustrates what Braket may return for a 2-qubit circuit, run with 100 shots.
from collections import Counter
counts = Counter({'00':53, '11':47})

This object can then be passed to other Tangelo functions right away. Below, we show how to compute an expectation value:

In [7]:
from tangelo.linq import get_expectation_value_from_frequencies_oneterm
from tangelo.linq.helpers.circuits.measurement_basis import pauli_string_to_of

# Rescale counts to obtain frequency histogram
freqs = {k:v/100 for k,v in counts.items()}
print(f"Frequencies: {freqs}")

# Compute expectation value for ZZ operator
exp_ZZ = get_expectation_value_from_frequencies_oneterm(pauli_string_to_of("ZZ"), freqs)
print(f"Expectation value for ZZ operator {exp_ZZ}")

Frequencies: {'00': 0.53, '11': 0.47}
Expectation value for ZZ operator 1.0
